Import all necessary libraries

In [18]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
import re
import json as js
# Import a scrapy Selector
!pip install scrapy
from scrapy import Selector
# Import requests
import requests
import pprint 
#!pip install flatten-dict
#from flatten_dict import flatten
#!pip install flatten_json
from pandas.io.json import json_normalize

Find 10~20 websites about this topic. Suggested ways to do this is
Define some keywords of this topic. Then, use google search to find a set of websites.
(Feel free to do anything fancier if you feel like doing it.)


In [0]:
# Do a google search.
# Optional: use the google search API: https://www.geeksforgeeks.org/performing-google-search-using-python-code/
# Optional: use the google custom search API: https://www.googleapis.com/customsearch/v1?
#url = 'https://www.google.com/search?q=iphone'

# Search Credit cards on Google / Bing
url = 'https://www.google.com/search?q=credit+cards'
# keyword to Search on Page DOM to extract sentences
keyword = "credit"
r = requests.get(url, allow_redirects=True)
import re
list_urls = re.findall("https://[^<>\"; ]+", str(r.content))
filtered_list_urls = [url for url in list_urls if 'google' not in url and 'gstatic' not in url]

all_sents = []
info_dict = {}
dict_column = ['Images','sentences', 'page_title', 'num_outgoing_links', 'website_last_modified' ]
img_ext = [ ".pdf", ".jpg", ".png", ".svg"]

#Function to check if the links on the website page hae image URL
def analyse_resource_extension(url):
    match = [ext in url for ext in img_ext]
    return any(element is True for element in match)

Part 1 (20 pts)
For each website, create a JSON object of the page that have

images (urls or the images themselves) 
sentences containing a few keywords.
outgoing links 
website last-modified (from header)



In [0]:
for url in filtered_list_urls[:50]:
    try:
        r = requests.get(url, allow_redirects=True)
        head_info = requests.head(url).headers
         # Create the string html containing the HTML source
        html = r.content
    except:
        continue
    # Optional: Try other setence extraction methods.
    extract_sent = re.findall(" [a-zA-Z ]+\.", str(r.content))
    extract_sents = [sent for sent in extract_sent if keyword in sent]
    # Create the Selector object sel from html
    sel = Selector( text = html )
    # Get the title of the website loaded in response
    page_title = sel.xpath('//title/text()').extract_first()
    # Get the last modified time from the header
    last_modified = head_info.get('last-modified')
    #find all thge outgoing links on the page
    all_links = re.findall("https://[^<>\"; ]+", str(r.content))
    #filter the subset of links which ends with image extension(invokes method defined above)
    links = list(filter(lambda x: analyse_resource_extension(x) is True, all_links))
    #creates a list of infomration tyo be added to disctonary
    site_data = [links, extract_sents,  page_title,len(all_links), last_modified]
    info_dict = dict(zip(dict_column, site_data))
    all_sents.append(info_dict)

Print The output as a list of the JSON objects. Just two JOSN Nodes

In [21]:
pp = pprint.PrettyPrinter(indent=4)
#Print First Two JSON Nodes.
pp.pprint(all_sents[:2])

[   {   'Images': [   'https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/img/facebook/facebook_share.png',
                      'https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/img/facebook/fb_ck-tile.jpg',
                      'https://ck-assets.imgix.net/assets/1.43.1/favicons/apple-touch-icon.png',
                      'https://ck-assets.imgix.net/assets/1.43.1/favicons/favicon-32x32.png',
                      'https://ck-assets.imgix.net/assets/1.43.1/favicons/favicon-32x32.png?h=16&w=16',
                      'https://creditkarmacdn-a.akamaihd.net/res/content/bundles/assets/1.43.1/favicons/safari-pinned-tab.svg',
                      "https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/images/browsers/browser-chrome.png\\'",
                      "https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/images/browsers/browser-firefox.png\\'",
                      "https://creditkarmacd

In [22]:
#Flatten the JSON and Normalize it by 'sentences' name the Column as below
df2 = json_normalize(all_sents,'sentences', [ 'Images', 'page_title', 'num_outgoing_links', 'website_last_modified' ], errors='ignore', record_prefix='sentences')
df2.fillna('?') 
df2.Images = df2.Images.apply(lambda y: np.nan if len(y)==0 else y)
new_index = df2['sentences0']
df3 = df2.set_index(new_index)
df3.to_csv('my_File.csv', sep = ',', na_rep = 'NA', columns=[ 'Images', 'page_title', 'num_outgoing_links', 'website_last_modified' ], header= True, index= True, index_label= 'Sentences')
#Print the sample value - Only for Printing Purpose. the Images Column here still shows empty list, but CSV doesn't have. I am continuing to use this as its formatteed well for human reading. 
json_normalize(all_sents,'sentences', [ 'Images', 'page_title', 'num_outgoing_links', 'website_last_modified' ], errors='ignore', record_prefix='sentences').fillna('?').head(20)

,sentences0,Images,page_title,num_outgoing_links,website_last_modified
0,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Scores & Credit Report Monitoring ...,71,?
1,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Scores & Credit Report Monitoring ...,71,?
2,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Score & Free Credit Reports With M...,82,?
3,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Score & Free Credit Reports With M...,82,?
4,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Score & Free Credit Reports With M...,82,?
5,credit scores should be free.,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Score & Free Credit Reports With M...,82,?
6,what affects your credit scores and what you ...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Score & Free Credit Reports With M...,82,?
7,personalized recommendations for ways to use ...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Score & Free Credit Reports With M...,82,?
8,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Scores & Credit Report Monitoring ...,72,?
9,Get the info you need to take control of your...,[https://creditkarmacdn-a.akamaihd.net/ckfiles...,Free Credit Scores & Credit Report Monitoring ...,72,?


In [23]:
#Load and read the exported csv
!cat 'my_File.csv'

Sentences,Images,page_title,num_outgoing_links,website_last_modified
 Get the info you need to take control of your credit.,"['https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/img/facebook/facebook_share.png', 'https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/img/facebook/fb_ck-tile.jpg', 'https://ck-assets.imgix.net/assets/1.43.1/favicons/apple-touch-icon.png', 'https://ck-assets.imgix.net/assets/1.43.1/favicons/favicon-32x32.png', 'https://ck-assets.imgix.net/assets/1.43.1/favicons/favicon-32x32.png?h=16&w=16', 'https://creditkarmacdn-a.akamaihd.net/res/content/bundles/assets/1.43.1/favicons/safari-pinned-tab.svg', ""https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/images/browsers/browser-chrome.png\\'"", ""https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/images/browsers/browser-firefox.png\\'"", ""https://creditkarmacdn-a.akamaihd.net/ckfiles.com/assets/428312337748/res/images/browser